In [ ]:
Avant de vous lancer dans la lecture de ce notebook, ouvrez un terminal dans jupyter (File -> New -> Terminal) et lancez les deux commandes ci-dessous :

    sudo apt update
    sudo apt install ffmpeg libavcodec-extra

# Whisper

De nombreux tutoriaux qui fleurissent :

- https://www.css.cnrs.fr/whisper-pour-retranscrire-des-entretiens/
- https://analyzingalpha.com/openai-whisper-python-tutorial

Intérêt ? 

- utiliser un modèle
- le faire en local

## Un coup d'oeil sur ce qu'est whisper

https://github.com/openai/whisper

## Installer la bibliothèque Python

Normalement toutes les librairies nécessaires au fonction de ce notebook (via le datalab d'Onyxia) sont installées. Toutefois, si le script d'initialisation n'a pas fonctionné vous pouvez "décommenter" la ligne ci-dessous. C'est-à-dire enlever le "#" avant de lancer la cellule.

In [2]:
#pip install openai-whisper

## Charger un modèle

In [15]:
import whisper
model1 = whisper.load_model("tiny")
model2 = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:29<00:00, 51.5MiB/s]


C'est un objet de type whisper

In [2]:
type(model1)

whisper.model.Whisper

## Modifier ses mp3

Créer un extrait de 30 s

In [7]:
#!pip install pydub audioop-lts

Note: you may need to restart the kernel to use updated packages.


In [8]:
#from pydub import AudioSegment
import pydub

In [9]:
itw = pydub.AudioSegment.from_mp3("250618_rdv_ref_donnee_led.mp3")

Extraire entre la seconde 30 et la seconde 90

In [10]:
extract = itw[30*1000:90*1000]

Ecrire sur le disque dur

In [12]:
pydub.AudioSegment.export(extract,"sample.mp3")

<_io.BufferedRandom name='sample.mp3'>

# Pour écouter l'extrait audio

In [22]:
from IPython.display import Audio, display

In [23]:
display(Audio('sample.mp3', autoplay=True))

## Retranscrire l'extrait

In [10]:
# Si jamais il y a une dépendance qui manque
#!pip install ffmpeg-python

Utiliser la fonction de transcription

In [13]:
result = model1.transcribe("sample.mp3")

/opt/python/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [14]:
result["text"]

" C'est parce que les couleurs sont à l'adile et donc on va avoir trois déclamistes. Ça donne des coups. Ce que vous pouvez pas l'inclouer, ce que vous faites dans l'amour. C'est un peu plus de plus de plus de plus de plus de plus de plus de plus de plus de plus. On peut produire. Ça va être forcément pas l'aide de nez, mais on a un temps de recherche de... Ça va virer un cas de feu et co-agétion comme si des gens se sont touchants. Ça va aller à terrorier, à terrorier, à... Moi, c'est un CV, c'est aussi le secteur. Il y a facilement ça, c'est juste de ne pas te laти. Il faut vraiment grounded auMUSO. Ça fait vraiment une fois. C'est très stable. On est en jusie d'ag Either Lloyd Dean officie, On va aller auMUSO. des choses qui m'ont offert très, je ne peux pas le flouisme de tête, je ne peux pas le flouisme de tête, je ne peux pas le flouisme de tête."

Essai d'un autre modèle plus précis

In [16]:
result = model2.transcribe("sample.mp3")
result["text"]

/opt/python/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


" Et là c'est le D, le laboratoire d'économie, c'est ça le Découp ? Est-ce que vous pouvez parler un peu de ce que vous faites dans le labo ? En termes de données qu'on produit... C'est un peu forcément parler de données, mais au moins en termes de recherche et de... Alors ça va varier, en face à l'éco-gestion, aussi des gens sont en co-gestion. Ça va aller de la théorique, on va dire, à du certainement statistique, les données sont individuelles, issues par exemple de décès ou apparentés, au niveau entreprise, etc. Donc ça, c'est plutôt des données qui préexistent. On va aller faire des données de marché social, des données de marché social, et enfin voilà. On va avoir des choses qui vont avoir un peu plus de tête..."

Mise sous la forme d'un tableau

In [17]:
import pandas as pd
speech = pd.DataFrame.from_dict(result['segments'])
speech.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.0,5.4,"Et là c'est le D, le laboratoire d'économie, ...","[50364, 3790, 3684, 269, 6, 377, 476, 413, 11,...",0.0,-0.495057,1.62844,0.213604
1,1,0,5.4,8.4,Est-ce que vous pouvez parler un peu de ce qu...,"[50634, 4410, 12, 384, 631, 2630, 18248, 16421...",0.0,-0.495057,1.62844,0.213604
2,2,0,8.4,12.4,En termes de données qu'on produit...,"[50784, 2193, 1433, 279, 368, 40101, 421, 6, 2...",0.0,-0.495057,1.62844,0.213604
3,3,0,12.4,16.4,"C'est un peu forcément parler de données, mai...","[50984, 383, 6, 377, 517, 5604, 31358, 16421, ...",0.0,-0.495057,1.62844,0.213604
4,4,0,16.4,21.4,"Alors ça va varier, en face à l'éco-gestion, ...","[51184, 9946, 2788, 2773, 1374, 811, 11, 465, ...",0.0,-0.495057,1.62844,0.213604


## Mise en forme et sauvegarde

(inspiration Yacine Chitour) 

In [18]:
def convertir(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

In [19]:
convertir(100000)

'27:46:40'

In [20]:
# Chemin où vous souhaitez enregistrer votre transcription d'entretien en format .txt
entretien_transcrit = "entretien.txt"
 
# Enregistrement de la transcription
with open(entretien_transcrit , 'w', encoding='utf-8') as f:
    for segment in result["segments"]:
        start_time = convertir(segment['start'])
        end_time = convertir(segment['end'])
        f.write(f"{start_time} - {end_time}: {segment['text']}\n")

In [21]:
print(open("entretien.txt","r").read())

00:00:00 - 00:00:05:  Et là c'est le D, le laboratoire d'économie, c'est ça le Découp ?
00:00:05 - 00:00:08:  Est-ce que vous pouvez parler un peu de ce que vous faites dans le labo ?
00:00:08 - 00:00:12:  En termes de données qu'on produit...
00:00:12 - 00:00:16:  C'est un peu forcément parler de données, mais au moins en termes de recherche et de...
00:00:16 - 00:00:21:  Alors ça va varier, en face à l'éco-gestion, aussi des gens sont en co-gestion.
00:00:22 - 00:00:28:  Ça va aller de la théorique, on va dire, à du certainement statistique,
00:00:28 - 00:00:34:  les données sont individuelles, issues par exemple de décès ou apparentés,
00:00:34 - 00:00:37:  au niveau entreprise, etc.
00:00:37 - 00:00:40:  Donc ça, c'est plutôt des données qui préexistent.
00:00:40 - 00:00:47:  On va aller faire des données de marché social, des données de marché social, et enfin voilà.
00:00:47 - 00:00:59:  On va avoir des choses qui vont avoir un peu plus de tête...



Tester l'erreur 

- https://github.com/jitsi/jiwer